In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# DistilBERT

In [2]:
! pip install --quiet lightning
! pip install --quiet transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.2/802.2 kB 21.8 MB/s eta 0:00:00


In [3]:
import pandas as pd
# importing all necessary packages
from os import listdir
from os.path import join
from sklearn.model_selection import train_test_split
import string
from torch.utils.data import Dataset, DataLoader
import torch
import torchmetrics
import pytorch_lightning as pl
from transformers import AutoModelForSequenceClassification
from pytorch_lightning.loggers import TensorBoardLogger, WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from transformers import DistilBertTokenizer

#### Dataset Preparation

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#show dataframe
train_data_preprocessed = pd.read_csv('/content/drive/MyDrive/NUS_MSBA/BT5153_Final Group Project_Shared Folder/Data/train_preprocessed_v2.csv')
train_data_preprocessed.head()

,text,label,%unique_word_total,%stop_word_total,mean_word_length,mean_char_count_per_word
0,school homework clubs become increasingly popu...,1.0,0.533742,0.411043,5.294479,6.294479
1,widely accepted knowledge great source power s...,1.0,0.491289,0.463415,5.125436,6.128920
2,first impressions great power shape interactio...,1.0,0.617647,0.475490,5.137255,6.137255
3,name address city state zip code email address...,1.0,0.432990,0.417526,5.395189,6.393471
4,limiting car usage numerous advantages benefit...,1.0,0.464876,0.409091,5.396694,6.394628


In [ ]:
train_data_preprocessed.shape

(28738, 6)

In [6]:
#show dataframe
test_data_preprocessed = pd.read_csv('/content/drive/MyDrive/NUS_MSBA/BT5153_Final Group Project_Shared Folder/Data/new_essay_val_preprocessed.csv')
test_data_preprocessed.head()

,text,label,%unique_word_total,%stop_word_total,mean_word_length,mean_char_count_per_word
0,marshall plan test progress test progress whet...,0,0.444112,0.392216,5.384232,6.384232
1,promoting global regional security postcold wa...,0,0.414807,0.424949,5.150101,6.150101
2,womanhood peacemaking taking advantage unity c...,0,0.475947,0.378710,5.583419,6.583419
3,human rights universal western construct cruci...,0,0.370482,0.503012,4.875502,5.875502
4,challenges american foreign service rebuilding...,0,0.467425,0.379524,5.668046,6.668046


In [7]:
from sklearn.model_selection import train_test_split
#show shape of train and test set
df_train, df_validation = train_test_split(train_data_preprocessed, test_size=0.3, random_state=42)
df_test = test_data_preprocessed # rename

print(df_train.shape)
print(df_validation.shape)
print(df_test.shape)
df_train.dropna(inplace = True)
df_validation.dropna(inplace = True)
df_test.dropna(inplace = True)
print(df_train.shape)
print(df_validation.shape)
print(df_test.shape)

(20116, 6)
(8622, 6)
(40, 6)
(20116, 6)
(8621, 6)
(40, 6)


In [8]:
# Prepare for DataLoader and Dataset without converting labels to ordinal
X_train_token_list = []
X_val_token_list = []
X_test_token_list = []
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Process the data
for i in df_train["text"]:
    X_train_token_list.append(tokenizer(i, truncation=True, padding='max_length', max_length=512, return_tensors='pt'))
for i in df_validation["text"]:
    X_val_token_list.append(tokenizer(i, truncation=True, padding='max_length', max_length=512, return_tensors='pt'))
for i in df_test["text"]:
    X_test_token_list.append(tokenizer(i, truncation=True, padding='max_length', max_length=512, return_tensors='pt'))


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [9]:
# add labels to token_list for use later as dataset
train_labels = []
for i in df_train["label"]:
  if int(i) == 0:
    train_labels.append(0)
  else:
    train_labels.append(1)
val_labels = []
for i in df_validation["label"]:
  if int(i) == 0:
    val_labels.append(0)
  else:
    val_labels.append(1)
test_labels = []
for i in df_test["label"]:
  if int(i) == 0:
    test_labels.append(0)
  else:
    test_labels.append(1)
for i in range(len(X_train_token_list)):
  X_train_token_list[i]["label"] = torch.tensor(train_labels[i])
for i in range(len(X_val_token_list)):
  X_val_token_list[i]["label"] = torch.tensor(val_labels[i])
for i in range(len(X_test_token_list)):
  X_test_token_list[i]["label"] = torch.tensor(test_labels[i])


In [10]:
# show our input data
print(X_train_token_list[1])

{'input_ids': tensor([[  101,  2116, 12637, 14879,  2482,  8192,  2451,  3032,  2605,  2762,
          7379,  2188,  3655,  2551,  2875,  6276,  2224,  3167, 19207,  2116,
          2111,  3182,  2424,  2478,  4522,  5193,  2965, 16481,  2625,  6911,
          3993,  2913,  2130,  2265,  8491,  4841, 13131,  3765,  3819,  2051,
          3693, 12436, 19761,  2078, 21240,  2047,  2259,  5247,  2625,  2051,
          3765,  2089,  4025,  2524,  2903,  2762,  2045,  2015,  9282,  2181,
          3901,  2444,  2302,  3765,  2429,  2446,  7575,  2166,  3632,  2302,
          3765, 12877, 29062,  4534, 12436, 19761,  2078,  2762,  3961,  2482,
         23301,  4998,  2270,  3665,  3720,  2163,  3963,  3867, 12436, 19761,
          3619,  2945,  3765,  5401,  3867,  2853,  2482,  2693,  2002,  3593,
         15532,  4787,  9339, 28142,  3038,  2482,  2467,  9049, 10047,  2172,
         19366,  2126,  4279,  2885,  2142,  2163,  5327,  2693,  2875,  2482,
         23301,  9580,  2344,  2468,  

In [11]:
# Create dataset and dataloader
class MyDataset(Dataset):
    def __init__(self, array_of_dicts):
        self.data = array_of_dicts

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        return item

trainset = MyDataset(X_train_token_list)
valset = MyDataset(X_val_token_list)
testset = MyDataset(X_test_token_list)

train_loader = DataLoader(
    dataset=trainset,
    batch_size=4,
    shuffle=True,
    num_workers=0
)

val_loader = DataLoader(
    dataset=valset,
    batch_size=4,
    num_workers=0
)

test_loader = DataLoader(
    dataset=testset,
    batch_size=4,
    num_workers=0
)

In [12]:
# train our model and freeze our parameter
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2)

for param in model.parameters():
    param.requires_grad = False

for param in model.pre_classifier.parameters():
    param.requires_grad = True

for param in model.classifier.parameters():
    param.requires_grad = True

#model structure
class LightningModel(pl.LightningModule):
    def __init__(self, model, learning_rate=5e-5):
        super().__init__()

        self.learning_rate = learning_rate
        self.model = model

        # Ensure the config is set
        self.config = model.config  # Copy the model's configuration

        self.train_acc = torchmetrics.Accuracy(task="multiclass", num_classes=2)
        self.train_auroc = torchmetrics.AUROC(task="multiclass", num_classes=2)  # AUROC for training


        self.val_acc = torchmetrics.Accuracy(task="multiclass", num_classes=2)
        self.val_auroc = torchmetrics.AUROC(task="multiclass", num_classes=2)  # AUROC for evaluation
        self.test_acc = torchmetrics.Accuracy(task="multiclass", num_classes=2)
        self.test_auroc = torchmetrics.AUROC(task="multiclass", num_classes=2)  # AUROC for testing

    def forward(self, input_ids, attention_mask, labels):
        return self.model(input_ids, attention_mask=attention_mask, labels=labels)

    def training_step(self, batch, batch_idx):
        outputs = self(batch["input_ids"].squeeze(1), attention_mask=batch["attention_mask"],
                       labels=batch["label"])
        self.log("train_loss", outputs["loss"])
        logits = outputs["logits"]
        probabilities = torch.softmax(logits, dim=1)  # Get probabilities for the positive class
        predicted_labels = torch.argmax(logits, 1)

        # Log accuracy
        self.train_acc(predicted_labels, batch["label"])
        self.log("train_acc", self.train_acc)

        # Update and log AUROC for training
        self.train_auroc(probabilities, batch["label"])
        # self.train_auroc.update(probabilities[:, 1], batch["label"])  # Use probabilities for the positive class
        self.log("train_auroc", self.train_auroc)

        return outputs["loss"]

    def validation_step(self, batch, batch_idx):
        outputs = self(batch["input_ids"].squeeze(1), attention_mask=batch["attention_mask"],
                       labels=batch["label"])
        self.log("val_loss", outputs["loss"], prog_bar=True)

        logits = outputs["logits"]
        probabilities = torch.softmax(logits, dim=1)  # Get probabilities
        predicted_labels = torch.argmax(logits, 1)

        # Log accuracy
        self.val_acc(predicted_labels, batch["label"])
        self.log("val_acc", self.val_acc, prog_bar=True)

        # Update and log AUROC
        self.val_auroc(probabilities, batch["label"])
        # self.val_auroc.update(probabilities[:, 1], batch["label"])  # Use probabilities for the positive class
        self.log("val_auroc", self.val_auroc, prog_bar=True)

        # output for classification report
        return {"logits": logits, "loss": outputs["loss"], "labels": batch["label"]}

    def test_step(self, batch, batch_idx):
        outputs = self(batch["input_ids"].squeeze(1), attention_mask=batch["attention_mask"],
                       labels=batch["label"])
        self.log("test_loss", outputs["loss"], prog_bar=True)

        logits = outputs["logits"]
        probabilities = torch.softmax(logits, dim=1)  # Get probabilities
        predicted_labels = torch.argmax(logits, 1)

        # Log accuracy
        self.test_acc(predicted_labels, batch["label"])
        self.log("accuracy", self.test_acc, prog_bar=True)

        # Update and log AUROC
        self.test_auroc(probabilities, batch["label"])
        self.log("test_auroc", self.test_auroc, prog_bar=True)

        # output for classification report
        return {"logits": logits, "loss": outputs["loss"], "labels": batch["label"]}

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer


lightning_model = LightningModel(model)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
# Change the monitor to "val_auroc"
logger = TensorBoardLogger("distilbert-add-fea/", name="finetuning", version="original")
callbacks = [
    ModelCheckpoint(
        save_top_k=1, mode="max", monitor="val_auroc"  # Monitor the AUROC metric
    )
]

In [14]:
trainer = pl.Trainer(
    max_epochs=3,
    callbacks=callbacks,
    # accelerator="gpu",  uncomment if gpu available
    devices=1,
    logger=logger,
    log_every_n_steps=1,
)

# Start training
trainer.fit(
    model=lightning_model,
    train_dataloaders=train_loader,
    val_dataloaders=val_loader   ###
)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name        | Type                                | Params
--------------------------------------------------------------------
0 | model       | DistilBertForSequenceClassification | 67.0 M
1 | train_acc   | MulticlassAccuracy                  | 0     
2 | train_auroc | MulticlassAUROC                     | 0     
3 | val_acc     | MulticlassAccuracy                  | 0     
4 | val_auroc   | MulticlassAUROC                     | 0     
5 | test_acc    | MulticlassAccuracy                  | 0     
6 | test_auroc  | MulticlassAUROC  

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=3` reached.


In [15]:
# Define the path to save the model
model_path = "/content/drive/MyDrive/NUS_MSBA/BT5153_Final Group Project_Shared Folder/models/distilbert/latest/distilbert-dataset-v2_ep3_dropout0.4.pth"

# Save the entire model
torch.save(lightning_model, model_path)

In [16]:
# test with validation dataset
val_results = trainer.validate(lightning_model, dataloaders=val_loader, ckpt_path="best")
val_results

INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at distilbert-add-fea/finetuning/original/checkpoints/epoch=2-step=15087.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at distilbert-add-fea/finetuning/original/checkpoints/epoch=2-step=15087.ckpt


Validation: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          val_acc          │    0.9250667095184326     │
│         val_auroc         │    0.9798523187637329     │
│         val_loss          │    0.19075165688991547    │
└───────────────────────────┴───────────────────────────┘

[{'val_loss': 0.19075165688991547,
  'val_acc': 0.9250667095184326,
  'val_auroc': 0.9798523187637329}]

In [17]:
test_results = trainer.test(lightning_model, dataloaders=test_loader, ckpt_path="best")
test_results

INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at distilbert-add-fea/finetuning/original/checkpoints/epoch=2-step=15087.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at distilbert-add-fea/finetuning/original/checkpoints/epoch=2-step=15087.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         accuracy          │    0.7250000238418579     │
│        test_auroc         │    0.8600000143051147     │
│         test_loss         │    0.5081993341445923     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.5081993341445923,
  'accuracy': 0.7250000238418579,
  'test_auroc': 0.8600000143051147}]

In [ ]:
# Start tensorboard.
%reload_ext tensorboard
%tensorboard --logdir distilbert/finetuning/original/

Reusing TensorBoard on port 6006 (pid 71933), started 2:54:01 ago. (Use '!kill 71933' to kill it.)